In [53]:
import scipy.optimize as opt
import numpy as np

In [54]:
class Pauli:
    
    def __init__(self):
        self.rules = [1,3,1,3]
        self.sign_rules = [[1,1,1,1], 
             [1, 1, 1j, -1j],
             [1, -1j, 1, 1j],
             [1, 1j, -1j, 1]]
        
        
    def product(self, a, b):
        p = (a + b*self.rules[a])%4
        return p
    
    
    def commutator(self, A, B):
    
        C = ()

        forwardsign = 1
        backwardsign = 1

        for i in range(len(A)):
            C += (self.product(A[i],B[i]),)
            forwardsign = forwardsign * self.sign_rules[A[i]][B[i]]
            backwardsign = backwardsign * self.sign_rules[B[i]][A[i]]

        if forwardsign == backwardsign:
            return 0

        else:
            return C
        
        
    def letters(self, A):
        
        label = {0: "I", 1: "X", 2: "Y", 3: "Z"}
        C=[]

        if A == []:

            pass
        
        elif type(A[0][1]) != tuple:
        
            for i in range(len(A)):
                B=""
                for j in range(len(A[i])):
                    B += label[A[i][j]]
                C.append(B)

        else:

            D=[]
            a=[]

            for i in range(len(A)):
                B=""
                a.append(A[i][0])
                for j in range(len(A[i][-1])):
                    B += label[A[i][-1][j]]
                D.append(B)
            
            C = list(zip(a,D))
            
        print(C)

In [31]:
class functions(Pauli):

    
    def __init__(self):
        Pauli.__init__(self)


    def mut_irr(self, n, x = np.pi):

        y = x%1
    
        if n > 1:
            return self.mut_irr(n-1, np.pi*y)
    
        else:
            return y


    def prepare(self, a, A):

        if type(A) == tuple:
            
            return [(a, A)]

        else:
            
            X = list(zip(a, A))
                
            return X

    
    def fullprod(self, Xp, Yp):

        prelist = []
        Alist = []
        Z = []
        
        for i in range(len(Xp)):
            for j in range(len(Yp)):

                A = ()
                prefactor = Xp[i][0] * Yp[j][0]
                
                for k in range(len(Xp[i][1])):
                    
                    A += (self.product(Xp[i][1][k], Yp[j][1][k]),)
                    prefactor = prefactor * self.sign_rules[Xp[i][1][k]][Yp[j][1][k]]

                if A not in Alist:
                    Alist.append(A)
                    prelist.append(prefactor)
                    

                else:
                    index = Alist.index(A)
                    prefactor += prelist[index]

                    if abs(prefactor) != 0:
                        prelist[index] = prefactor

                    else:
                        Alist.pop(index)
                        prelist.pop(index)

        Zp = list(zip(prelist, Alist))

        return Zp

    
    def euler(self, a, A):
        
        I = (0,)*len(A)
        Xp = self.prepare(np.cos(a), I)
        Yp = self.prepare(1j*np.sin(a), A)

        return Xp + Yp 

    
    def KXK(self, a, K_r, Xp):

        b = a if type(a) == list else [a]
        K = K_r if type(K_r) == list else [K_r]
        Zp = Xp

        for i in range(len(K)):
        
            Kp = self.euler(b[-(1+i)], K[-(1+i)])
            Kdagp = self.euler(-b[-(1+i)], K[-(1+i)])
            Yp = self.fullprod(Kp, Zp)
            Zp = self.fullprod(Yp, Kdagp)

        return Zp 

    
    def trace(self, X):
        t = 0
    
        for i in range(len(X)):
    
            if X[i][1] == (0,)*len(X[i][1]):
                t += X[i][0]
    
        return t

In [49]:
class Hamiltonians:
    
    
    def __init__(self, N, model = None):
        self.N = N
        self.model = model
        
        
    def Hamiltonian(self, coeff = None):
    
        if self.model == 'TFIM':
            H = []
            p = []
            x = [1]*self.N if coeff == None else coeff
            

            for i in range(self.N-1):

                l = [0]*self.N
                l[i] = 1
                l[i+1] = 1
                H.append(tuple(l))
                p.append(1)

                l = [0]*self.N
                l[i] = 3
                H.append(tuple(l))
                p.append(x[i])

            l = [0]*self.N
            l[self.N-1] = 3
            H.append(tuple(l))
            p.append(x[-1])

            return H, p
        
           
        elif self.model == 'XY':
            H = []
            p = [1]*2*(self.N-1)
            #l = [0]*self.N

            for i in range(self.N-1):

                l = [0]*self.N
                l[i] = 1
                l[i+1] = 1
                H.append(tuple(l))
                
                l = [0]*self.N
                l[i] = 2
                l[i+1] = 2
                H.append(tuple(l))

            return H, p
        
        
        elif self.model == 'TFXY':
            H = []
            p = []
            x = [1]*self.N if coeff == None else coeff

            for i in range(self.N-1):

                l = [0]*self.N
                l[i] = 1
                l[i+1] = 1
                H.append(tuple(l))
                p.append(1)
                
                l = [0]*self.N
                l[i] = 2
                l[i+1] = 2
                H.append(tuple(l))
                p.append(1)

                l = [0]*self.N
                l[i] = 3
                H.append(tuple(l))
                p.append(x[i])

            l = [0]*self.N
            l[self.N-1] = 3
            H.append(tuple(l))
            p.append(x[-1])

            return H, p
        
        
        elif self.model == 'TFXYY':
            H = []
            #l = [0]*self.N

            for i in range(self.N-1):

                l = [0]*self.N
                l[i] = 1
                l[i+1] = 1
                H.append(tuple(l))
                
                l = [0]*self.N
                l[i] = 2
                l[i+1] = 2
                H.append(tuple(l))
                
                l = [0]*self.N
                l[i] = 1
                l[i+1] = 2
                H.append(tuple(l))

                l = [0]*self.N
                l[i] = 3
                H.append(tuple(l))

            l = [0]*self.N
            l[self.N-1] = 3
            H.append(tuple(l))

            return H
        
        
        elif self.model == 'Heisenberg':
            H = []
            p = [1]*3*(self.N-1)
            #l = [0]*self.N

            for i in range(self.N-1):

                l = [0]*self.N
                l[i] = 1
                l[i+1] = 1
                H.append(tuple(l))
                
                l = [0]*self.N
                l[i] = 2
                l[i+1] = 2
                H.append(tuple(l))

                l = [0]*self.N
                l[i] = 3
                l[i+1] = 3
                H.append(tuple(l))
                
            return H, p
        
        
        elif self.model == 'CFXY':
            H = []
            p = []
            x = [(1,1)]*self.N if coeff == None else coeff

            for i in range(self.N-1):

                l = [0]*self.N
                l[i] = 1
                l[i+1] = 1
                H.append(tuple(l))
                p.append(1)
                
                l = [0]*self.N
                l[i] = 2
                l[i+1] = 2
                H.append(tuple(l))
                p.append(1)

                l = [0]*self.N
                l[i] = 3
                H.append(tuple(l))
                p.append(x[i][0])
                
                l = [0]*self.N
                l[i] = 2
                H.append(tuple(l))
                p.append(x[i][1])

            l = [0]*self.N
            l[self.N-1] = 3
            H.append(tuple(l))
            p.append(x[-1][0])
            
            l = [0]*self.N
            l[self.N-1] = 2
            H.append(tuple(l))
            p.append(x[-1][1])

            return H, p


        elif self.model == 'UCC':

            H = [(1,1,1,3,3,2), (2,2,1,3,3,2), (1,2,2,3,3,2), (2,1,2,3,3,2), (1,2,1,3,3,1), (2,1,1,3,3,1), (1,1,2,3,3,1), (2,2,2,3,3,1),
                 (1,1,0,1,2,0), (2,2,0,1,2,0), (1,2,0,1,1,0), (2,1,0,1,1,0), (1,2,0,2,2,0), (2,1,0,2,2,0), (1,1,0,2,1,0), (2,2,0,2,1,0)]

            return H, None
   

        else:
            
            return []
    
    
    def algebra(self):
        
        g = self.Hamiltonian()[0]
        s = [-1]*len(g)
        finalindex = len(g) - 1
        initialindex = -1
        t = True
        cont = False
        
        while t == True:
            t = False
            
            for i in range(finalindex, initialindex, -1):
                for j in range(i-1, -1, -1):
                    Com = Pauli().commutator(g[i],g[j])
                    sign = s[i]*s[j]

                    if Com != 0:
                        
                        if Com not in g:
                            g.append(Com)
                            s.append(sign)
                            t = True
                            
                        elif sign != s[g.index(Com)]:
                            cont = True
                            
                        
                        
            initialindex = finalindex
            finalindex = len(g) - 1
            
        return g, s, cont

In [50]:
class Cartan(Hamiltonians):


    def __init__(self, N, model = None):
        Hamiltonians.__init__(self, N, model)

    
    
    def decomposition(self):

        k = []
        m = []
        A = self.algebra()


        if not A[2]:

            for i in range(len(A[0])):

                if A[1][i] == 1:
                    k.append(A[0][i])

                else:
                    m.append(A[0][i])

        else:
        
            k = []
            m = []
            #c = []
            
            for i in range(len(A)):
                if (A[0][i].count(2))%2 != 0:
                    k.append(A[0][i])
                    
                else:
                    m.append(A[0][i])
                    #c.append(A[i].count(0))
                    
        return k, m#, np.argmax(c)
    
    
    def subalgebra(self):
        
        m = self.decomposition()[1]
        c = []

        for i in range(len(m)):
            c.append(self.N - m[i].count(0))

        m_sorted = [x for _,x in sorted(zip(c, m))]

            
        h = [m_sorted[0]]
        
        for i in range(1,len(m_sorted)):
            for j in range(len(h)):
                
                if Pauli().commutator(m_sorted[i],h[j]) != 0:
                    break
                    
                elif j == len(h)-1:
                    h.append(m_sorted[i])
                    
        return h

In [52]:
Pauli().letters(Cartan(4, "UCC").decomposition()[0])
Pauli().letters(Cartan(4, "UCC").subalgebra())

['IZXXYX', 'ZIXXYX', 'IZYXYY', 'ZIYXYY', 'IZYXXX', 'ZIYXXX', 'IZXXXY', 'ZIXXXY', 'IZYYYX', 'ZIYYYX', 'IZXYYY', 'ZIXYYY', 'IZXYXX', 'ZIXYXX', 'IZYYXY', 'ZIYYXY']
['XXIXYI', 'XXIYXI', 'XXXIIY', 'XXYIIX', 'YYIXYI', 'YYIYXI', 'YYXIIY', 'YYYIIX', 'XXXZZY', 'XXYZZX', 'XXZXYZ', 'XXZYXZ', 'YYXZZY', 'YYYZZX', 'YYZXYZ', 'YYZYXZ']


In [6]:
class parameters(Cartan, functions):

    
    def __init__(self, N, model = None):
        Cartan.__init__(self, N, model)
        functions.__init__(self)


    def find(self, coeff = None):

        H = self.Hamiltonian(coeff)
        Hp = self.prepare(H[1], H[0])
        K = self.decomposition()[0]
        h = self.subalgebra()
        gamma = [self.mut_irr(i+1) for i in range(len(h))]
        hp = self.prepare(gamma, h)
        #bound = np.pi

        def f_r(a):

            Xp = self.KXK([a[i] for i in range(len(K))], K, hp)
            Fp = self.fullprod(Xp, Hp)
            return np.real(self.trace(Fp))

        a0 = 2*np.pi*np.random.rand(len(K))

        #bounds = ((-bound, bound),)*len(a0)

        result = opt.minimize(f_r, a0, method="BFGS")

        a = result.x % (2*np.pi)

        c = list(-a)
        c.reverse()
        K.reverse()

        Hp = self.KXK(c, K, Hp)

        i = 0
        while i < len(Hp):

            if abs(Hp[i][0]) < 1e-4:
                Hp.pop(i)

            else:
                i += 1

        return a, Hp

In [8]:
p = parameters(4, "TFIM").find(coeff = None)
angles = p[0]
Hamiltonian = p[1]
print(angles)
Pauli().letters(Hamiltonian)

[1.75886096 1.2816594  0.62676318 2.76111691 2.77565727 3.38819292
 4.84331193 5.97625216 3.36116364 1.47700004 1.85993322 1.02989171]
[((-1.5320888862379196+0j), 'IZII'), ((-0.9999999999996051+0j), 'ZIII'), ((-0.3472963553341023+0j), 'IIIZ'), ((-1.879385241571621+0j), 'IIZI')]
CPU times: user 6 s, sys: 0 ns, total: 6 s
Wall time: 6 s


In [8]:
model = Cartan(4, "TFIM")
print("4-site TFIM \n ")
print("H: ")
Pauli().letters(model.Hamiltonian()[0])
print("\ng: ")
Pauli().letters(model.algebra()[0])
print("\nk: ")
Pauli().letters(model.decomposition()[0])
print("\nm: ")
Pauli().letters(model.decomposition()[1])
print("\nh: ")
Pauli().letters(model.subalgebra())

4-site TFIM 
 
H: 
['XXII', 'ZIII', 'IXXI', 'IZII', 'IIXX', 'IIZI', 'IIIZ']

g: 
['XXII', 'ZIII', 'IXXI', 'IZII', 'IIXX', 'IIZI', 'IIIZ', 'IIXY', 'IIYX', 'IXYI', 'IYXI', 'XYII', 'YXII', 'YZXI', 'YYII', 'XZYI', 'XZXI', 'IYZX', 'IYYI', 'IXZY', 'IXZX', 'IIYY', 'XZZY', 'YZZY', 'IYZY', 'YZZX', 'XZZX', 'YZYI']

k: 
['IIXY', 'IIYX', 'IXYI', 'IYXI', 'XYII', 'YXII', 'YZXI', 'XZYI', 'IYZX', 'IXZY', 'XZZY', 'YZZX']

m: 
['XXII', 'ZIII', 'IXXI', 'IZII', 'IIXX', 'IIZI', 'IIIZ', 'YYII', 'XZXI', 'IYYI', 'IXZX', 'IIYY', 'YZZY', 'IYZY', 'XZZX', 'YZYI']

h: 
['IIIZ', 'IIZI', 'IZII', 'ZIII']
